In [1]:
import json
import os
import pickle

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [2]:
with open("/openai/api_key.json") as f:
    config = json.load(f)


os.environ["OPENAI_API_KEY"] = config["key"]

In [3]:
SYSTEM_MESSAGE = SystemMessage("""
You are an expert in language and knowledgeable on how words are used.

Your task is to generate as many diverse definitions as possible for the a given word.  Follow these steps:

1. come up with a list of all possible parts of speech that the given word can be,e.g., noun, verb, adjective, etc.
2. for each part of speech, generate one or more examples of the given word for that parts of speech.  preappend the part of speech to the examples, e.g., "noun: example1", "verb: example2", etc.
3. combine all examples into a single list.

Return your response as a JSON object with the word as the key and the connotations as a list of strings.

example:

{
  "word": [
    "noun: example1", 
    "noun: example2", 
    "adjective: example3",]
}

""")

In [4]:
def ask_llm_for_solution(prompt, model="gpt-4o", temperature=0.7, max_tokens=4096):
   
    # Initialize the OpenAI LLM with your API key and specify the GPT-4o model
    llm = ChatOpenAI(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        model_kwargs={"response_format": {"type": "json_object"}},
    )


    conversation = [SYSTEM_MESSAGE, prompt]

    # Invoke the LLM
    response = llm.invoke(conversation)

    return response


In [5]:
words = ["uphold", "discard", "honor", "energy", "state", "play", "justice", "labor", "pass", "fulfill", "draw", "keep", "blanket", "sham", "sheet", "throw"]

vocabulary = {}

for the_word in words:
    prompt = f"\n\ngiven word: {the_word}"
    prompt = HumanMessage(prompt)
    result = ask_llm_for_solution(prompt, temperature=1.0, max_tokens=4096)
    vocabulary[the_word] = json.loads(result.content)[the_word]

for word, definitions in vocabulary.items():
    print(f"\n\n{word}:")
    for definition in definitions:
        print(definition)




uphold:
verb: to support or maintain something, such as a law, decision, or principle, even in the face of opposition.
verb: to confirm or approve a judgment or decision.
verb: to physically hold something up and prevent it from falling.


discard:
verb: to get rid of something because it is no longer useful or desirable
verb: to throw away or dispose of something
verb: to abandon or renounce something, such as a plan or idea
noun: something that has been thrown away or disposed of
noun: a card that is thrown out in a card game


honor:
noun: high respect or esteem
noun: a privilege or an award given in recognition of achievement
noun: adherence to a conventional standard of conduct
verb: to regard or treat with admiration and respect
verb: to fulfill an obligation or keep a promise
verb: to confer distinction upon someone or something
noun: a person's sense of self-worth or pride
verb: to show courteous behavior or politeness towards someone


energy:
noun: The capacity to do work o

In [6]:
# Save the vocabulary to a file
with open("vocabulary.pkl", "wb") as f:
    pickle.dump(vocabulary, f)